## NLP with Machine Learning

### 1. Sentiment Analysis

In [1]:
import pandas as pd

# create a list of sentences
data = [
    "When life gives you lemons, make lemonade! 🙂",
    "She bought 2 lemons for $1 at Maven Market.",
    "A dozen lemons will make a gallon of lemonade. [AllRecipes]",
    "lemon, lemon, lemons, lemon, lemon, lemons",
    "He's running to the market to get a lemon — there's a great sale today.",
    "iced tea is my favorite",
    "I didn't like the taste of that lemonade at all.",
    "My lemons went bad before I could use them, unfortunately.",
]

# expand the column width to see the full sentences
pd.set_option('display.max_colwidth', None)

# turn it into a dataframe
data_df = pd.DataFrame(data, columns=["sentence"])
data_df.head()

# make a copy of the dataframe
df = data_df.copy()
df.head()

,sentence
0,"When life gives you lemons, make lemonade! 🙂"
1,She bought 2 lemons for $1 at Maven Market.
2,A dozen lemons will make a gallon of lemonade. [AllRecipes]
3,"lemon, lemon, lemons, lemon, lemon, lemons"
4,He's running to the market to get a lemon — there's a great sale today.


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
test = df.sentence[0]
test

'When life gives you lemons, make lemonade! 🙂'

In [4]:
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(test)

{'neg': 0.0, 'neu': 0.75, 'pos': 0.25, 'compound': 0.4587}

In [5]:
analyzer.polarity_scores(test)['compound'] # Total Score.

0.4587

In [6]:
def get_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    return analyzer.polarity_scores(text)['compound'] # Total Score.

In [7]:
df['sentiment'] = df.sentence.apply(get_sentiment)

In [8]:
df

,sentence,sentiment
0,"When life gives you lemons, make lemonade! 🙂",0.4587
1,She bought 2 lemons for $1 at Maven Market.,0.0000
2,A dozen lemons will make a gallon of lemonade. [AllRecipes],0.0000
3,"lemon, lemon, lemons, lemon, lemon, lemons",0.0000
4,He's running to the market to get a lemon — there's a great sale today.,0.6249
5,iced tea is my favorite,0.4588
6,I didn't like the taste of that lemonade at all.,-0.2755
7,"My lemons went bad before I could use them, unfortunately.",-0.7096


### 2. Text Classification

#### GOAL: Predict which reviews are high priority (vs low priority) that we need to address right away

In [9]:
# import libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [10]:
reviews = pd.read_excel('Data/Popchip_Reviews.xlsx')
reviews.head()

,Id,UserId,Rating,Priority,Title,Text
0,23689,A21SYVGVNG8RAS,5,Low,Yummy snacks!,Popchips are the bomb!! I use the parmesan garlic to scoop up cottage cheese as a healthy alternative to chips and dip. My healthy eating program is saved.
1,23690,AQJYXC0MPRQJL,5,Low,Great chip that is different from the rest,"I like the puffed nature of this chip that makes it more unique in the chip market. I ordered the Salt and Vinegar and absolutely love that flavor, hands down my favorite chip ever. I have tried the cheddar and regular flavors as well. The cheddar is about a 4/5 and the regular is about a 3/5 because I prefer strong flavors and obviously that would not be the case for the regular. The Salt and Vinegar is kind of weak compared to some regular S&V chips, but is quite flavorful and makes you wanting to come back for more."
2,23691,A30NYUHEDLWI0Y,5,Low,Great Alternative to Potato Chips,"I just love these chips! I was always a big fan of potato chips, but haven't had one since I discovered popchips. They are great for dipping or all alone. I am constantly re-ordering them. One note however-if you are on a low salt diet these chips are probably not for you. They are high in sodium. We go through a case every two months. If you love them it pays to join the subscribe and save program through Amazon. You save money and stay supplied!"
3,23692,A2NU55U9LKTB5J,3,High,Not somthing I would crave,"These tasted like potatoe stix, that we got in grade school with our lunches usually on pizza day. They were the bomb then, not so much now. Won't buy again unless I get them for cheap or free."
4,23693,A225F7QFP5LIW2,5,Low,healthy and delicious,"These chips are great! They look almost like a flattened rice cake, but taste so much better, more like a potato chip. The bbq flavor is delicious. They are very low in fat and full of flavor. It is easy to eat an entire bag of these!"


In [11]:
reviews.shape

(564, 6)

In [12]:
reviews.Priority.value_counts()

Priority
Low     447
High    117
Name: count, dtype: int64

In [13]:
import maven_text_preprocessing

In [14]:
reviews['Text_Clean'] = maven_text_preprocessing.clean_and_normalize(reviews.Text)
reviews.head(2)

,Id,UserId,Rating,Priority,Title,Text,Text_Clean
0,23689,A21SYVGVNG8RAS,5,Low,Yummy snacks!,Popchips are the bomb!! I use the parmesan garlic to scoop up cottage cheese as a healthy alternative to chips and dip. My healthy eating program is saved.,popchip bomb use parmesan garlic scoop cottage cheese healthy alternative chip dip healthy eat program save
1,23690,AQJYXC0MPRQJL,5,Low,Great chip that is different from the rest,"I like the puffed nature of this chip that makes it more unique in the chip market. I ordered the Salt and Vinegar and absolutely love that flavor, hands down my favorite chip ever. I have tried the cheddar and regular flavors as well. The cheddar is about a 4/5 and the regular is about a 3/5 because I prefer strong flavors and obviously that would not be the case for the regular. The Salt and Vinegar is kind of weak compared to some regular S&V chips, but is quite flavorful and makes you wanting to come back for more.",like puff nature chip make unique chip market order salt vinegar absolutely love flavor hand favorite chip try cheddar regular flavor cheddar 45 regular 35 prefer strong flavor obviously case regular salt vinegar kind weak compare regular sv chip flavorful make want come


In [15]:
# cv, naive bayes

In [16]:
cv = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.2)
X = cv.fit_transform(reviews.Text_Clean)
X_df = pd.DataFrame(X.toarray(), columns=cv.get_feature_names_out())
X_df

,bag,buy,calorie,chip,eat,flavor,good,great,like,love,popchip,potato,potato chip,salt,snack,taste,try
0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,4,0,3,0,0,1,1,0,0,0,2,0,0,1
2,0,0,0,3,0,0,0,1,0,2,1,1,1,1,0,0,0
3,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
4,1,0,0,2,1,2,0,1,2,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,0,0,0,3,3,1,1,5,0,1,1,4,3,0,0,1,0
560,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,1
561,0,0,0,2,0,1,0,2,0,0,0,0,0,0,0,2,0
562,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0


In [17]:
y = reviews.Priority # Y value for the model
y.head()

0     Low
1     Low
2     Low
3    High
4     Low
Name: Priority, dtype: object

In [18]:
# train/test split
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)

# model
model_nb = MultinomialNB()
model_nb.fit(X_train, y_train)

# predict
y_pred_nb = model_nb.predict(X_test)

# evaluate
print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

Accuracy: 0.8407079646017699
              precision    recall  f1-score   support

        High       0.60      0.16      0.25        19
         Low       0.85      0.98      0.91        94

    accuracy                           0.84       113
   macro avg       0.73      0.57      0.58       113
weighted avg       0.81      0.84      0.80       113



In [19]:
# test it out on new reviews
new_reviews = pd.Series([
    "Pop chips are my favorite! I love these chips so much.",
    "Taste bad. I don't like the flavor options or taste.",
    "Solid snack."
])

new_reviews

0    Pop chips are my favorite! I love these chips so much.
1      Taste bad. I don't like the flavor options or taste.
2                                              Solid snack.
dtype: object

In [20]:
new_reviews_clean = maven_text_preprocessing.clean_and_normalize(new_reviews)
new_reviews_df = pd.DataFrame(cv.transform(new_reviews_clean).toarray(), columns=cv.get_feature_names_out())
new_reviews_df

,bag,buy,calorie,chip,eat,flavor,good,great,like,love,popchip,potato,potato chip,salt,snack,taste,try
0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,2,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [21]:
model_nb.predict(new_reviews_df)

array(['Low', 'High', 'Low'], dtype='<U4')

In [22]:
# Second round of modeling.
# tfidf, logistic regression

In [23]:
tv = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.2)
Xt = tv.fit_transform(reviews.Text_Clean)
Xt_df = pd.DataFrame(Xt.toarray(), columns=tv.get_feature_names_out())
Xt_df

,bag,buy,calorie,chip,eat,flavor,good,great,like,love,popchip,potato,potato chip,salt,snack,taste,try
0,0.000000,0.000000,0.0,0.392603,0.656435,0.000000,0.000000,0.000000,0.000000,0.000000,0.644170,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.0,0.561185,0.000000,0.537701,0.000000,0.000000,0.195524,0.213766,0.000000,0.000000,0.000000,0.513094,0.000000,0.000000,0.220814
2,0.000000,0.000000,0.0,0.517908,0.000000,0.000000,0.000000,0.295101,0.000000,0.526082,0.283255,0.277355,0.333330,0.315684,0.000000,0.000000,0.000000
3,0.000000,0.690063,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.512918,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.510616,0.000000
4,0.252776,0.000000,0.0,0.340747,0.284866,0.435318,0.000000,0.291234,0.474884,0.000000,0.000000,0.273721,0.328962,0.000000,0.000000,0.236376,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,0.000000,0.000000,0.0,0.216106,0.361330,0.092028,0.103897,0.615680,0.000000,0.109758,0.118193,0.462925,0.417263,0.000000,0.000000,0.099942,0.000000
560,0.381673,0.000000,0.0,0.000000,0.000000,0.328649,0.000000,0.439742,0.000000,0.000000,0.422089,0.000000,0.000000,0.000000,0.459181,0.000000,0.404891
561,0.000000,0.000000,0.0,0.399843,0.000000,0.255407,0.000000,0.683486,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.554742,0.000000
562,0.000000,0.000000,0.0,0.000000,0.000000,0.537244,0.606536,0.000000,0.586074,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
y.head()

0     Low
1     Low
2     Low
3    High
4     Low
Name: Priority, dtype: object

In [25]:
# train/test split
Xt_train, Xt_test, yt_train, yt_test = train_test_split(Xt_df, y, test_size=0.2, random_state=42)

# model
model_lr = LogisticRegression()
model_lr.fit(Xt_train, yt_train)

# predict
y_pred_lr = model_lr.predict(Xt_test)

# evaluate
print("Accuracy:", accuracy_score(yt_test, y_pred_lr))
print(classification_report(yt_test, y_pred_lr))

Accuracy: 0.8407079646017699
              precision    recall  f1-score   support

        High       1.00      0.05      0.10        19
         Low       0.84      1.00      0.91        94

    accuracy                           0.84       113
   macro avg       0.92      0.53      0.51       113
weighted avg       0.87      0.84      0.78       113



In [26]:
reviews.head(2)

,Id,UserId,Rating,Priority,Title,Text,Text_Clean
0,23689,A21SYVGVNG8RAS,5,Low,Yummy snacks!,Popchips are the bomb!! I use the parmesan garlic to scoop up cottage cheese as a healthy alternative to chips and dip. My healthy eating program is saved.,popchip bomb use parmesan garlic scoop cottage cheese healthy alternative chip dip healthy eat program save
1,23690,AQJYXC0MPRQJL,5,Low,Great chip that is different from the rest,"I like the puffed nature of this chip that makes it more unique in the chip market. I ordered the Salt and Vinegar and absolutely love that flavor, hands down my favorite chip ever. I have tried the cheddar and regular flavors as well. The cheddar is about a 4/5 and the regular is about a 3/5 because I prefer strong flavors and obviously that would not be the case for the regular. The Salt and Vinegar is kind of weak compared to some regular S&V chips, but is quite flavorful and makes you wanting to come back for more.",like puff nature chip make unique chip market order salt vinegar absolutely love flavor hand favorite chip try cheddar regular flavor cheddar 45 regular 35 prefer strong flavor obviously case regular salt vinegar kind weak compare regular sv chip flavorful make want come


In [27]:
reviews['Predictions_NB'] = model_nb.predict_proba(X_df)[:, 0]

In [28]:
reviews['Predictions_LR'] = model_lr.predict_proba(Xt_df)[:, 0]

In [29]:
reviews.sort_values(by='Predictions_NB', ascending=False).head()

,Id,UserId,Rating,Priority,Title,Text,Text_Clean,Predictions_NB,Predictions_LR
550,24239,A2ZKS33N6Y3EPC,3,High,"Taste more like ""Tomato and Basil"" than ""Chili and Lime""","NOTE: This review is for the Chili and Lime Flavor Popchip. Amazon had a separate page for it but then merged the product and its reviews into one.<br /><br />It's hard to objectively review food since everyone's palate and tastes are different. So what I can say about this particular Popchip flavor that should be useful for most folks out there is that it doesn't really taste like Chili and Lime you're ""probably"" expecting. The Chili and Lime most folks probably are expecting if they grew up on Frito Lay products is very sharp and sweet (and of course artificial) - but it's what we liked if we ate more than a bag.<br /><br />The best way I can describe this flavor is that it has a ""tomato"" like taste to it with a somewhat tangy sour note that is suppose to be the lime component. Together they turn into an odd combination that registers other flavors in your mind than Chili and Lime - at least it did to me and others who tasted it with me. If you eat the skin of a green bell pepper, you can kind of get at what Popchips were trying to do with the Chili taste on this version, but I have no idea how some sour salt can be akin to lime. For myself personally, I thought it tasted like ""Tomato and Basil"" you would find on Pita chip flavors and baked snacks.<br /><br />Whether or not you agree with my above description of the flavor, I would highly suggest you try to get this in a sample pack and try it out first. BBQ + Salt & Vinegar Popchips are still my staples for now.",note review chili lime flavor popchip amazon separate page merge product review onebr br hard objectively review food everyone palate taste different particular popchip flavor useful folk not taste like chili lime probably expect chili lime folk probably expect grow frito lay product sharp sweet course artificial like eat bagbr br good way describe flavor tomato like taste somewhat tangy sour note suppose lime component turn odd combination register flavor mind chili lime taste eat skin green bell pepper kind popchip try chili taste version idea sour salt akin lime personally think taste like tomato basil find pita chip flavor bake snacksbr br agree description flavor highly suggest try sample pack try bbq salt vinegar popchip staple,0.973989,0.478529
96,23785,AE5AHEH3NLPBZ,3,High,Tastes Like Celery,"I really like pop chips, but this flavor isn't the best. I was expecting these to taste like chili peppers and lime (Spicy, Sweet, and Tart), but instead of going for a chili pepper taste, they went for a chili the food taste. This wouldn't be so bad, except they taste overwhelmingly of tomato and celery. The reason they didn't call them Tomato and Celery Chips is because it is sounds gross and no one would buy that, and unfortunately it tastes like it sounds.",like pop chip flavor not good expect taste like chili pepper lime spicy sweet tart instead go chili pepper taste go chili food taste not bad taste overwhelmingly tomato celery reason not tomato celery chip sound gross buy unfortunately taste like sound,0.854032,0.495605
463,24152,A2ZMMQ4W17EK2N,2,High,Original PopChips,"Bought the Original flavor from the store and just tried them tonight. They were very greasy and salty. I did not like them a lot. I will not purchase this original flavor again. However I can't complain because I got the 3 ounce bag for only $1.00 at the store while they were on sale. I tried the BBq flavor and they are delicious. I bought the sea salt & vinegar, and cheddar but haven't tried those yet.",buy original flavor store try tonight greasy salty like lot purchase original flavor not complain get 3 ounce bag 100 store sale try bbq flavor delicious buy sea salt vinegar cheddar not try,0.760037,0.439521
21,23710,ASIMCC20UVK58,5,Low,Great Chips Less Fat,"I eat chips almost every day and decided I wanted to find s

### 3. Topic Modeling

**GOAL: Find the main themes in the reviews**

In [34]:
tv2 = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.2)
Xt2 = tv2.fit_transform(reviews.Text_Clean)
Xt_df2 = pd.DataFrame(Xt2.toarray(), columns=tv2.get_feature_names_out())
Xt_df2

,100,alternative,amazon,bad,bake,baked,bbq,big,bit,box,...,thing,think,time,variety,ve,vinegar,want,way,weight,work
0,0.0,0.465515,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.348295,0.193511,0.000000,0.000000,0.000000
2,0.0,0.000000,0.354088,0.000000,0.0,0.000000,0.000000,0.428869,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.354475,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,0.0,0.324462,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.299888,0.0,0.000000,0.000000,0.000000,0.337388,0.657147,0.000000
560,0.0,0.000000,0.190702,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.197896,0.000000,0.208227,0.000000,0.000000,0.247474
561,0.0,0.378621,0.000000,0.380993,0.0,0.396437,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
562,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.543142,0.000000,0.000000,0.000000,0.000000,0.000000


In [84]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=5, random_state=42, max_iter=500)
W = nmf.fit_transform(Xt_df2) # documents-topics
H = nmf.components_ # topics-terms

In [85]:
# H.shape

In [86]:
# H[0]

In [87]:
def display_topics(H, num_words=10):
    for topic_num, topic_array in enumerate(H):
        top_features = topic_array.argsort()[::-1][:num_words]
        top_words = [tv2.get_feature_names_out()[i] for i in top_features]
        print("Topic", topic_num+1, ":", ', '.join(top_words))

In [88]:
display_topics(H)

Topic 1 : order, amazon, case, time, store, box, thing, price, know, product
Topic 2 : sweet, salty, br, light, rice, texture, think, little, crunchy, fry
Topic 3 : healthy, alternative, bbq, delicious, regular, feel, work, enjoy, nice, look
Topic 4 : br, vinegar, bbq, favorite, pepper, original, lime, think, sea, sour
Topic 5 : fat, low, weight, pop, regular, serve, diet, crunch, single, tasty


In [89]:
reviews.head(2)

,Id,UserId,Rating,Priority,Title,Text,Text_Clean,Predictions_NB,Predictions_LR
0,23689,A21SYVGVNG8RAS,5,Low,Yummy snacks!,Popchips are the bomb!! I use the parmesan garlic to scoop up cottage cheese as a healthy alternative to chips and dip. My healthy eating program is saved.,popchip bomb use parmesan garlic scoop cottage cheese healthy alternative chip dip healthy eat program save,0.200606,0.259019
1,23690,AQJYXC0MPRQJL,5,Low,Great chip that is different from the rest,"I like the puffed nature of this chip that makes it more unique in the chip market. I ordered the Salt and Vinegar and absolutely love that flavor, hands down my favorite chip ever. I have tried the cheddar and regular flavors as well. The cheddar is about a 4/5 and the regular is about a 3/5 because I prefer strong flavors and obviously that would not be the case for the regular. The Salt and Vinegar is kind of weak compared to some regular S&V chips, but is quite flavorful and makes you wanting to come back for more.",like puff nature chip make unique chip market order salt vinegar absolutely love flavor hand favorite chip try cheddar regular flavor cheddar 45 regular 35 prefer strong flavor obviously case regular salt vinegar kind weak compare regular sv chip flavorful make want come,0.305999,0.228199


In [92]:
doc_topics = pd.DataFrame(W)
doc_topics.columns = ['orders', 'taste & texture', 'good', 'flavor', 'health']
doc_topics

,orders,taste & texture,good,flavor,health
0,0.000000,0.000000,0.403012,0.000000,0.000000
1,0.055080,0.000000,0.023755,0.115179,0.088048
2,0.067787,0.000000,0.000000,0.000000,0.153890
3,0.017647,0.002463,0.000000,0.000000,0.029204
4,0.000000,0.016166,0.040860,0.044669,0.190659
...,...,...,...,...,...
559,0.025953,0.010370,0.050308,0.000000,0.168847
560,0.108660,0.000000,0.022080,0.157261,0.032282
561,0.084727,0.000000,0.200482,0.000000,0.091203
562,0.019073,0.000000,0.000000,0.085505,0.037631


In [96]:
reviews_topics = pd.concat([reviews.Text, doc_topics], axis=1)
reviews_topics

,Text,orders,taste & texture,good,flavor,health
0,Popchips are the bomb!! I use the parmesan garlic to scoop up cottage cheese as a healthy alternative to chips and dip. My healthy eating program is saved.,0.000000,0.000000,0.403012,0.000000,0.000000
1,"I like the puffed nature of this chip that makes it more unique in the chip market. I ordered the Salt and Vinegar and absolutely love that flavor, hands down my favorite chip ever. I have tried the cheddar and regular flavors as well. The cheddar is about a 4/5 and the regular is about a 3/5 because I prefer strong flavors and obviously that would not be the case for the regular. The Salt and Vinegar is kind of weak compared to some regular S&V chips, but is quite flavorful and makes you wanting to come back for more.",0.055080,0.000000,0.023755,0.115179,0.088048
2,"I just love these chips! I was always a big fan of potato chips, but haven't had one since I discovered popchips. They are great for dipping or all alone. I am constantly re-ordering them. One note however-if you are on a low salt diet these chips are probably not for you. They are high in sodium. We go through a case every two months. If you love them it pays to join the subscribe and save program through Amazon. You save money and stay supplied!",0.067787,0.000000,0.000000,0.000000,0.153890
3,"These tasted like potatoe stix, that we got in grade school with our lunches usually on pizza day. They were the bomb then, not so much now. Won't buy again unless I get them for cheap or free.",0.017647,0.002463,0.000000,0.000000,0.029204
4,"These chips are great! They look almost like a flattened rice cake, but taste so much better, more like a potato chip. The bbq flavor is delicious. They are very low in fat and full of flavor. It is easy to eat an entire bag of these!",0.000000,0.016166,0.040860,0.044669,0.190659
...,...,...,...,...,...,...
559,"I love potato chips. I could eat them by the bagful but thanks to the powers that be, this would not provide anyone with enough nutrition to survive. Nonetheless I have eaten my share of potato chips over the years, and perhaps as a result I have been watching my weight lately. I discovered these great popchips and they hit the spot in a number of ways. First they are a low-calorie alternative to regular potato chips, meaning I can eat a normal-sized portion and still keep on target with my weight loss goals. Second, they are gluten-free, which is great for those of us who have gluten intolerance issues. Third, they taste great - light and airy, crispy, rich in that great potato flavor that keeps me reaching for another one. This is a great product and I hope it stays around for a good long time.",0.025953,0.010370,0.050308,0.000000,0.168847
560,"When PopChips were really hard to find, I was ordering them by the case from Amazon on a regular basis. The price was always great and these really are my go-to snack. They are always fresh and delicious from Amazon.<br /><br />I definitely prefer Original over barbecue and sour cream & onion. I haven't tried other flavors, but Original works just fine for me because I use it for various dips and will even use dry seasonings on them if I'm in the mood.<br /><br />I don't know anybody that I've offered some of these to who didn't want a bag of their own! Highly recommended.",0.108660,0.000000,0.022080,0.157261,0.032282
561,These are a much healthy alternative to most chips and they taste great. They have a great crunch and flavor and don't have that bad after taste that most baked chips have. My only regret is that I didn't order more when they were on sale there all gone now but even at regular price are worth it.,0.084727,0.000000,0.200482,0.000000,0.091203
562,"These are so good, I've started getting them automatically. I like the original flavor, but they have a lot of others.",0.019073,0.000000,0.000000,0.085505,0.037631


In [97]:
# Combine: Topic, Sentiment, EDA

In [98]:
final_topics = reviews_topics.copy()
final_topics.head(2)

,Text,orders,taste & texture,good,flavor,health
0,Popchips are the bomb!! I use the parmesan garlic to scoop up cottage cheese as a healthy alternative to chips and dip. My healthy eating program is saved.,0.00000,0.0,0.403012,0.000000,0.000000
1,"I like the puffed nature of this chip that makes it more unique in the chip market. I ordered the Salt and Vinegar and absolutely love that flavor, hands down my favorite chip ever. I have tried the cheddar and regular flavors as well. The cheddar is about a 4/5 and the regular is about a 3/5 because I prefer strong flavors and obviously that would not be the case for the regular. The Salt and Vinegar is kind of weak compared to some regular S&V chips, but is quite flavorful and makes you wanting to come back for more.",0.05508,0.0,0.023755,0.115179,0.088048


In [103]:
final_topics['top_topic'] = final_topics.iloc[:, 1:].idxmax(axis=1)
final_topics

,Text,orders,taste & texture,good,flavor,health,top_topic
0,Popchips are the bomb!! I use the parmesan garlic to scoop up cottage cheese as a healthy alternative to chips and dip. My healthy eating program is saved.,0.000000,0.000000,0.403012,0.000000,0.000000,good
1,"I like the puffed nature of this chip that makes it more unique in the chip market. I ordered the Salt and Vinegar and absolutely love that flavor, hands down my favorite chip ever. I have tried the cheddar and regular flavors as well. The cheddar is about a 4/5 and the regular is about a 3/5 because I prefer strong flavors and obviously that would not be the case for the regular. The Salt and Vinegar is kind of weak compared to some regular S&V chips, but is quite flavorful and makes you wanting to come back for more.",0.055080,0.000000,0.023755,0.115179,0.088048,flavor
2,"I just love these chips! I was always a big fan of potato chips, but haven't had one since I discovered popchips. They are great for dipping or all alone. I am constantly re-ordering them. One note however-if you are on a low salt diet these chips are probably not for you. They are high in sodium. We go through a case every two months. If you love them it pays to join the subscribe and save program through Amazon. You save money and stay supplied!",0.067787,0.000000,0.000000,0.000000,0.153890,health
3,"These tasted like potatoe stix, that we got in grade school with our lunches usually on pizza day. They were the bomb then, not so much now. Won't buy again unless I get them for cheap or free.",0.017647,0.002463,0.000000,0.000000,0.029204,health
4,"These chips are great! They look almost like a flattened rice cake, but taste so much better, more like a potato chip. The bbq flavor is delicious. They are very low in fat and full of flavor. It is easy to eat an entire bag of these!",0.000000,0.016166,0.040860,0.044669,0.190659,health
...,...,...,...,...,...,...,...
559,"I love potato chips. I could eat them by the bagful but thanks to the powers that be, this would not provide anyone with enough nutrition to survive. Nonetheless I have eaten my share of potato chips over the years, and perhaps as a result I have been watching my weight lately. I discovered these great popchips and they hit the spot in a number of ways. First they are a low-calorie alternative to regular potato chips, meaning I can eat a normal-sized portion and still keep on target with my weight loss goals. Second, they are gluten-free, which is great for those of us who have gluten intolerance issues. Third, they taste great - light and airy, crispy, rich in that great potato flavor that keeps me reaching for another one. This is a great product and I hope it stays around for a good long time.",0.025953,0.010370,0.050308,0.000000,0.168847,health
560,"When PopChips were really hard to find, I was ordering them by the case from Amazon on a regular basis. The price was always great and these really are my go-to snack. They are always fresh and delicious from Amazon.<br /><br />I definitely prefer Original over barbecue and sour cream & onion. I haven't tried other flavors, but Original works just fine for me because I use it for various dips and will even use dry seasonings on them if I'm in the mood.<br /><br />I don't know anybody that I've offered some of these to who didn't want a bag of their own! Highly recommended.",0.108660,0.000000,0.022080,0.157261,0.032282,flavor
561,These are a much healthy alternative to most chips and they taste great. They have a great crunch and flavor and don't have that bad after taste that most baked chips have. My only regret is that I didn't order more when they were on sale there all gone now but even at regular price are worth it.,0.084727,0.000000,0.200482,0.000000,0.091203,good
562,"These are so good, I've started getting them automatically. I like the original flavor, but they have a lot of others.",0.019073,0.000000,0.000000,0.085505,0.037631,flavor


In [ ]:
def get_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    return analyzer.polarity_scores(text)['compound'] # Total Score.

In [106]:
final_topics['sentiment'] = final_topics.Text.apply(get_sentiment)
final_topics.head(2)

,Text,orders,taste & texture,good,flavor,health,top_topic,sentiment
0,Popchips are the bomb!! I use the parmesan garlic to scoop up cottage cheese as a healthy alternative to chips and dip. My healthy eating program is saved.,0.00000,0.0,0.403012,0.000000,0.000000,good,0.9244
1,"I like the puffed nature of this chip that makes it more unique in the chip market. I ordered the Salt and Vinegar and absolutely love that flavor, hands down my favorite chip ever. I have tried the cheddar and regular flavors as well. The cheddar is about a 4/5 and the regular is about a 3/5 because I prefer strong flavors and obviously that would not be the case for the regular. The Salt and Vinegar is kind of weak compared to some regular S&V chips, but is quite flavorful and makes you wanting to come back for more.",0.05508,0.0,0.023755,0.115179,0.088048,flavor,0.7269


In [108]:
final_topics.groupby('top_topic')['sentiment'].mean().sort_values()

top_topic
orders             0.504758
health             0.711142
flavor             0.768537
good               0.816834
taste & texture    0.842701
Name: sentiment, dtype: float64